In [1]:
%pip install sktime==0.11.3
#%pip install numpy=1.20

In [2]:
import numpy as np
import pandas as pd
import timeit
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
import tensorflow as tf
from pickle import dump
from sklearn.model_selection import KFold

from sktime.classification.dictionary_based import BOSSEnsemble
from sktime.datasets import load_from_arff_to_dataframe as load_arff
#numpy.version.version

In [3]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

In [7]:
kf = KFold(n_splits=10, shuffle=True, random_state=1)
#cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
X, y = load_arff("AllCandidas_TRAIN_V2.arff")
print(np.shape(X))
print(type(X))
precision = []
accuracy = []
f1 = []
recall = []

def gpu(X, y, kf):
  with tf.device('/device:GPU:0'):
    i= 0
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]     
        BOSS = BOSSEnsemble(max_ensemble_size=3,min_window =1, max_win_len_prop=2)
        print(f"Interaction {i}")
        print(f"In trainning...")
        BOSS.fit(X_train, y_train)
        print(f"In prediction...")
        i=i+1
        # Predict and print accuracy
        predictions = BOSS.predict(X_test)
        precision.append(precision_score(y_test, predictions, average='macro',zero_division=1))
        accuracy.append(accuracy_score(y_test, predictions))
        recall.append(recall_score(y_test, predictions, average='macro',zero_division=1))
        f1.append(f1_score(y_test, predictions, average='macro'))
        #print("In prediction...")
        # Predict and print accuracy
       # predictions = HC2.predict(X_test)
    print(type(X_train))
    print(np.shape(X_train))
    print(f"Precision: {np.mean(precision)}, Accuracy:{np.mean(accuracy)}, Recall: {np.mean(recall)}, F1: {np.mean(f1)}")
    return np.mean(precision), np.mean(accuracy), np.mean(recall), np.mean(f1), np.std(accuracy), BOSS

(12256, 1)
<class 'pandas.core.frame.DataFrame'>


In [8]:
if tf.config.list_physical_devices('GPU'):
  strategy = tf.distribute.MirroredStrategy()
else:  # Use the Default Strategy
  strategy = tf.distribute.get_strategy()

with strategy.scope():
  # Do something interesting
  print(tf.Variable(1.))
  precision, accuracy, recall, f1, std, BOSS = gpu(X,y,kf)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
MirroredVariable:{
  0: <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.0>
}
Interaction 0
In trainning...
In prediction...
Interaction 1
In trainning...
In prediction...
Interaction 2
In trainning...
In prediction...
Interaction 3
In trainning...
In prediction...
Interaction 4
In trainning...
In prediction...
Interaction 5
In trainning...
In prediction...
Interaction 6
In trainning...
In prediction...
Interaction 7
In trainning...
In prediction...
Interaction 8
In trainning...
In prediction...
Interaction 9
In trainning...
In prediction...
<class 'pandas.core.frame.DataFrame'>
(11031, 1)
Precision: 0.6869951186138294, Accuracy:0.6838265472583813, Recall: 0.684700363097488, F1: 0.6830747756418953


In [9]:
df_metrics = pd.DataFrame(data=np.zeros((1, 5), dtype=np.float), index=[0],
                       columns=['precision', 'accuracy', 'recall','f1_score','std'])
df_metrics['precision'] = precision
df_metrics['accuracy'] = accuracy
df_metrics['recall'] = recall
df_metrics['f1_score'] = f1
df_metrics['std'] = std
df_metrics

C:\Users\mlb\AppData\Local\Temp/ipykernel_13960/3617186161.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df_metrics = pd.DataFrame(data=np.zeros((1, 5), dtype=np.float), index=[0],


,precision,accuracy,recall,f1_score,std
0,0.686995,0.683827,0.6847,0.683075,0.018313


In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import fbeta_score, make_scorer, precision_score

def gpu():
  with tf.device('/device:GPU:0'):
    res = pd.DataFrame(data=np.zeros((1, 4), dtype=np.float), index=[0],
                       columns=['precision', 'accuracy', 'recall','f1_score'])
    X_train, y_train = load_arff("AllCandidas_TEST_Data.arff")
    cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
    precision = make_scorer(precision_score, average='weighted',zero_division=1)
    recall = make_scorer(recall_score, average='weighted',zero_division=1)

    print(type(X_train))
    print(type(y_train))
    print(np.shape(X_train))
    print(np.shape(y_train))

    #Creating BOSSEnsemble
    BOSS = BOSSEnsemble(max_ensemble_size=3,min_window =1, max_win_len_prop=2)
    print("In prediction...")
    precision_result = cross_val_score(BOSS, X_train, y_train, cv=cv,scoring=precision).mean()
    print(f"Precision result: {precision_result}")
    res['precision'] = precision_result 
    
    res['accuracy'] = cross_val_score(BOSS, X_train, y_train, cv=cv, scoring='accuracy').mean()
    print(f"Accuracy result: {res['accuracy'][0]}")
    res['recall'] = cross_val_score(BOSS, X_train, y_train, cv=cv,scoring=recall).mean()
    print(f"Recall result: {res['recall'][0]}")
    res['f1_score'] = cross_val_score(BOSS, X_train, y_train, cv=cv,scoring='f1_weighted').mean()
    print(f"F1_score result: {res['f1_score'][0]}")

    return BOSS , res
BOSS_model, res = gpu()
res

In [11]:
#save metrics
print("Saving metrics...")
df_metrics.to_csv('results/'+ 'df_metrics_BOSS.csv', index=False)
#save the model
print("Saving model...")
dump(BOSS, open('results/'+'BOSS_model.pkl', 'wb'))
print("Metrics of model:")
df_metrics

Saving metrics...
Saving model...
Metrics of model:


,precision,accuracy,recall,f1_score,std
0,0.686995,0.683827,0.6847,0.683075,0.018313


In [ ]:
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))